In [1]:
import numpy as np
import cv2
import pytesseract
from pdf2image import convert_from_path

pytesseract.pytesseract.tesseract_cmd = "C:/Users/pawvaibh/AppData/Local/Programs/Tesseract-OCR/tesseract.exe"

pdf_path = "C:/Users/pawvaibh/Downloads/chapter07.pdf"

pages = convert_from_path(pdf_path, 350)

page_texts = []


for page in pages:
    
    np_array = np.array(page)
    
    rgb_image = cv2.cvtColor(np_array, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY)
   
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 30)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    
    dilate = cv2.dilate(thresh, kernel, iterations=4)
    
    cnts, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

   
    line_items_coordinates = []   
    for c in cnts:
        area = cv2.contourArea(c)
        if area > 10000:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(rgb_image, (x, y), (x + w, y + h), (36, 255, 12), 3)
            line_items_coordinates.append([(x,y), (x + w, y + h)])
 
    page_text = ""
    for i in range(len(line_items_coordinates) -1, -1, -1):
       
        c = line_items_coordinates[i]
        roi = rgb_image[c[0][1]:c[1][1], c[0][0]:c[1][0]]    
        ret, thresh1 = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    
        text = pytesseract.image_to_string(thresh1, config='--psm 6')
        page_text += text.strip() + "\n"

    page_texts.append(page_text)

for i, text in enumerate(page_texts):
    print(f"Page {i+1} Text:")
    print(text)

Page 1 Text:
Chapter 7 Newton’s Laws of Motion
7.1 Force and Quantity of Matter...............sscccsssssssssssccsssssssssccsssssssesscssosssssssscscssseee I
Example 7.1 Vector Decomposition Solution ................ccssssscccscssssssseessccsssssseesssssees O
7.1.1 Mass Calibrationn.............cccvccccsssccsssccssvcccsscccescccccescccssccsencccescccsescccessccsenscceescces F
7.2 Newton ’s First Law...........cccvccocvccccssscccssccenscccescccecccccesccensccsencccesecccescccessccssssceeeeees
7.3 Momentum, Newton’s Second Law and Third Law. ..........cccccccsssssssssscesssssssssssees O
7.4 Newton’s Third Law: Action-Reaction Pairs............cccccscccsssccesscccesscccssscccsscceencces 7
© Peter Dourmashkin 2012

Page 2 Text:
Chapter 7 Newton’s Laws of Motion
I have not as yet been able to discover the reason for these properties of
gravity from phenomena, and I do not feign hypotheses. For whatever is
not deduced from the phenomena must be called a hypothesis; and
hypotheses, whether met